In [1]:
import re
from itertools import islice
from collections import defaultdict
import datetime

import os
import json

import warnings
warnings.filterwarnings("ignore")

import pandas as pd

import numpy as np

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
ru_stopwords = stopwords.words('russian')

from bertopic import BERTopic

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import ParameterGrid

from sklearn.metrics import recall_score, precision_score, f1_score

from sentence_transformers import SentenceTransformer

import umap

from lib.semconvtree import (
    Geogrid,
    TimePeriodRange,
    GeoTimeAggregator,
    SemConvTree
)

from utils import apply_clean, get_text_and_hashtags

SEED = 42

sentence_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [2]:
# замена функций для вычисления косинусного расстояния, потому что 
# в библиотеке pynndescent, которая используется в bertopic, была проблема
# с библиотекой numpy или где-то ещё

import numba

numba.set_num_threads(8)

@numba.njit(fastmath=True)
def correct_alternative_cosine(ds):
    result = np.empty_like(ds)
    for i in range(ds.shape[0]):
        result[i] = 1.0 - np.power(2.0, ds[i])
    return result

import pynndescent
pynn_dist_fns_fda = pynndescent.distances.fast_distance_alternatives
pynn_dist_fns_fda["cosine"]["correction"] = correct_alternative_cosine
pynn_dist_fns_fda["dot"]["correction"] = correct_alternative_cosine

In [16]:
vk_posts_df = pd.read_csv('jkh_groups_data/jkh_posts2_processed.csv')
vk_posts_df2 = pd.read_csv('jkh/jkh_posts2_new_processed.csv')
vk_posts_df2.columns = ['Unnamed: 0', 'post_id', 'owner_id', 'date', 'text', 'clean_text', 'similar', 'neg_sent','has_phone', 'from_big_group']

vk_posts_df = pd.concat([vk_posts_df, vk_posts_df2]).drop_duplicates(['post_id', 'owner_id'])

vk_posts_df.drop(columns=['Unnamed: 0'], inplace=True)
not_empty = vk_posts_df[vk_posts_df['clean_text'].str.strip().str.len() > 0]
filtered = not_empty[((not_empty['similar'] == 1) | (not_empty['neg_sent'] == 1)) & (~not_empty['has_phone']) & (~not_empty['from_big_group'])]
print(len(filtered))
filtered.head()

4049


,date,post_id,owner_id,text,clean_text,similar,neg_sent,has_phone,from_big_group
10,2018-11-23 00:00:00,33,-116263595,Добрый день. Напротив второй парадной у нас оч...,добрый день. напротив второй парадной у нас оч...,1,0,False,False
49,2017-04-15 00:00:00,2269,-26534225,Середина апреля 2017 года наступила ...... Где...,середина апреля 2017 года наступила ...... где...,0,1,False,False
52,2017-05-15 00:00:00,2296,-26534225,В преддверии годового собрания ТСЖ хотелось бы...,в преддверии годового собрания тсж хотелось бы...,0,1,False,False
53,2017-05-17 00:00:00,2300,-26534225,2.Миф о невозможности заменить трубы - эта зим...,2.миф о невозможности заменить трубы - эта зим...,0,1,False,False
54,2017-05-20 00:00:00,2307,-26534225,3. Результаты суда. На самом деле суд указал н...,3. результаты суда. на самом деле суд указал н...,0,1,False,False


In [17]:
vk_groups_df = pd.read_csv('jkh_groups_data/vk_groups_addresses2.csv')
vk_groups_df = vk_groups_df.drop_duplicates(['id'])
vk_groups_df_with_address = vk_groups_df[vk_groups_df.lat.notna()]
vk_groups_df_with_address

,id,name,description,members_count,name_clean,description_clean,shortname_tsj_clean,address_house_clean,address_tsj,name_tsj,...,name_tsj_clean,address_house,clusterCaption_house,lat_house,lon_house,clusterCaption_house_clean,city,street,house,coord_source
0,117404680,"ТСЖ ""Просвещения 53-1 литер. Д""",Приглашаем в группу неравнодушных жильцов наше...,46,ТСЖ Просвещения 53-1 литер. Д,Приглашаем в группу неравнодушных жильцов наше...,"ТСЖ Просвещения 53-1, литер д","пр-кт Просвещения, 41 Литер А","Санкт-Петербург г, пр-кт. Просвещения, д. 53, ...",Товарищество собственников жилья «Просвещения ...,...,Товарищество собственников жилья Просвещения 5...,"пр-кт Просвещения, 41 Литер А","пр-кт Просвещения, 41 Литер А",60.047245,30.359553,"пр-кт Просвещения, 41 Литер А",санкт-петербург,NaN,NaN,NaN
1,166797525,"ТСЖ ""Академическое""",NaN,15,ТСЖ Академическое,NaN,ТСЖ Академическое,NaN,"Санкт-Петербург г, пр-кт. Гражданский, д. 79, ...",Товарищество собственников жилья «Академическое»,...,Товарищество собственников жилья Академическое,NaN,NaN,NaN,NaN,NaN,санкт-петербург,NaN,NaN,NaN
4,90003929,"ТСЖ ""РЖЕВКА"" пр. Косыгина 33 корп. 1","группа создана для обслуждения предложений, пр...",3,ТСЖ РЖЕВКА пр. Косыгина 33 корп. 1,"группа создана для обслуждения предложений, пр...",NaN,"пр-кт Косыгина, 33 корпус 1 Литер А",NaN,NaN,...,NaN,"пр-кт Косыгина, 33 корпус 1 Литер А","пр-кт Косыгина, 33 корпус 1 Литер А",59.944499,30.499430,"пр-кт Косыгина, 33 корпус 1 Литер А",санкт-петербург,ТСЖ РЖЕВКА пр. Косыгина 33 корп. 1,NaN,NaN
5,166842485,ТСЖ Коммуны 48 лит.А,Эта группа для всех собственников и проживающи...,15,ТСЖ Коммуны 48 лит.А,Эта группа для всех собственников и проживающи...,NaN,"ул. Коммуны, 48 Литер А",NaN,NaN,...,NaN,"ул. Коммуны, 48 Литер А","ул. Коммуны, 48 Литер А",59.958444,30.490788,"ул. Коммуны, 48 Литер А",санкт-петербург,Эта группа для всех собственников и проживающи...,д.48,NaN
7,172336349,ТСЖ Ростовская 5 корпус 2 (Славянка),Официальная группа собственников жилья дома 5 ...,34,ТСЖ Ростовская 5 корпус 2 Славянка,Официальная группа собственников жилья дома 5 ...,NaN,"ул. Брюсовская, 5 корпус 2 Литер А",NaN,NaN,...,NaN,"ул. Брюсовская, 5 корпус 2 Литер А","ул. Брюсовская, 5 корпус 2 Литер А",59.987312,30.423980,"ул. Брюсовская, 5 корпус 2 Литер А",санкт-петербург,Официальная группа собственников жилья дома 5 ...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3143,32581894,"Круглый стол «Технологии АНТИЛЕД: дороги, тран...","В пятницу, 25 ноября 2011 года, в 12.30 МСК в ...",2,"Круглый стол Технологии АНТИЛЕД: дороги, транс...","В пятницу, 25 ноября 2011 года, в 12.30 МСК в ...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,санкт-петербург,дороги,д 2011,city_street_house
3144,32911093,"ЖК Ленинградские вечера, ул. Мебельная д. 47 к.1",Главная площадка собственников и дольщиков дом...,723,"ЖК Ленинградские вечера, ул. Мебельная д. 47 к.1",Главная площадка собственников и дольщиков дом...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,санкт-петербург,ул. Мебельная д. 47 к.1,д. 47,city_street_house
3145,43601429,5 курс 080507.65(0611)ПЛМиД ИРЭиУ з/о,Институт региональной экономики и управления -...,4,5 курс 080507.650611ПЛМиД ИРЭиУ з/о,Институт региональной экономики и управления -...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,санкт-петербург,Институт региональной экономики и управления -...,д.15,city_street_house
3146,43819015,"Народный патруль. Б. Сампсониевский пр. 108, Сер","Народный Патруль. Б. Самсониевский пр., д.108 ...",5,"Народный патруль. Б. Сампсониевский пр. 108, Сер","Народный Патруль. Б. Самсониевский пр., д.108 ...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,санкт-петербург,Сампсониевский пр. 108,д.108,city_street_house


In [18]:
groups_ids = -vk_groups_df_with_address['id']
filtered.owner_id.isin(groups_ids).sum()

2835

In [19]:
filtered[filtered.owner_id.isin(groups_ids)].to_csv('jkh_groups_data/candidates_posts.csv', index=None)

In [20]:
with open('jkh_groups_data/groups_ids_with_addresses.txt', 'w') as f:
    f.write('\n'.join(map(str, groups_ids.tolist())))

# Играюсь с размеченными в gpt-3.5 данными

Часть данных была размечена с помощью gpt-3.5, по сути использовалась как дополнительная фильтрация постов на наличие проблемы.

Все посты, которые прошли все шаги фильтрации, используются при тематическом моделировании с помощью bertopic.

In [21]:
posts_df = pd.read_csv('jkh_groups_data/candidates_posts_labelled.csv', sep='|')
jkh_posts_df = posts_df[posts_df['label'] == 1]

groups_temp_df = vk_groups_df_with_address[['id', 'lat', 'lon']]
groups_temp_df['id'] = -groups_temp_df['id']

jkh_posts_df = jkh_posts_df.merge(groups_temp_df, left_on='owner_id', right_on='id', how='left')

jkh_posts_df

,date,post_id,owner_id,text,clean_text,similar,neg_sent,has_phone,from_big_group,label,id,lat,lon
0,2018-11-23 00:00:00,33,-116263595,Добрый день. Напротив второй парадной у нас оч...,добрый день. напротив второй парадной у нас оч...,1,0,False,False,1,-116263595,59.938955,30.315644
1,2017-04-15 00:00:00,2269,-26534225,Середина апреля 2017 года наступила ...... Где...,середина апреля 2017 года наступила ...... где...,0,1,False,False,1,-26534225,60.012125,30.333951
2,2017-05-17 00:00:00,2300,-26534225,2.Миф о невозможности заменить трубы - эта зим...,2.миф о невозможности заменить трубы - эта зим...,0,1,False,False,1,-26534225,60.012125,30.333951
3,2017-05-20 00:00:00,2307,-26534225,3. Результаты суда. На самом деле суд указал н...,3. результаты суда. на самом деле суд указал н...,0,1,False,False,1,-26534225,60.012125,30.333951
4,2017-05-23 00:00:00,2309,-26534225,4. Открытая информация и обсуждение всех! вопр...,4. открытая информация и обсуждение всех! вопр...,0,1,False,False,1,-26534225,60.012125,30.333951
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1668,2021-09-22 20:18:28,208,-70134067,сумасшедший на уровне 9-10-11этажа 3й - 2й пар...,сумасшедший на уровне 9-10-11этажа 3й - 2й пар...,0,1,False,False,1,-70134067,59.937540,30.350687
1669,2020-12-10 09:51:26,129,-64635732,Уважаемые автовладельцы! \nДавайте не будем св...,уважаемые автовладельцы! давайте не будем свин...,0,1,False,False,1,-64635732,59.938955,30.315644
1670,2021-04-12 12:41:40,173,-8210550,Открытие детского сада на 220 мест на ул. Черк...,открытие детского сада на 220 мест на ул. черк...,0,1,False,False,1,-8210550,60.030721,30.426199
1671,2009-03-23 08:05:20,3,-8210550,"Да уж.....\nБеседа остановилась на том, что ad...","да уж..... беседа остановилась на том, что adm...",0,1,False,False,1,-8210550,60.030721,30.426199


In [22]:
jkh_docs = jkh_posts_df.clean_text.tolist()
jkh_embeddings = sentence_model.encode(jkh_docs, show_progress_bar=False)

In [23]:
vectorizer_model = CountVectorizer(stop_words=ru_stopwords, ngram_range=(1, 2))
topic_model_jkh_posts = BERTopic(language="multilingual", vectorizer_model=vectorizer_model, min_topic_size=6, calculate_probabilities=True)

In [24]:
topics, probs = topic_model_jkh_posts.fit_transform(jkh_docs, jkh_embeddings)

In [25]:
topic_model_jkh_posts.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,504,-1_это_дома_тсж_соседи,"[это, дома, тсж, соседи, нашего, нам, сегодня,...","[дорогие друзья, жители дома! эта группа созда..."
1,0,279,0_воды_горячей_вода_воду,"[воды, горячей, вода, воду, холодной, горячей ...","[кто нибудь знает, почему горячей воды нет?, г..."
2,1,138,1_отопление_квитанции_батареи_отопления,"[отопление, квитанции, батареи, отопления, руб...","[добрый день! елена, подскажите пожалуйста, в ..."
3,2,77,2_ремонт_парадной_это_стены,"[ремонт, парадной, это, стены, ремонта, работы...",[у нас в прошлом году в 7-й парадной выше 5 эт...
4,3,48,3_света_свет_освещение_работает,"[света, свет, освещение, работает, домофон, эт...",[пожар в грщ-4 был! я звонила сегодня в аварий...
5,4,41,4_лифт_лифты_работают_лифте,"[лифт, лифты, работают, лифте, лифтов, вообще,...",[снова лифты работают с проблемами.средний про...
6,5,39,5_дворе_площадки_грязь_это,"[дворе, площадки, грязь, это, детской, качели,...",[дорогие соседи! уборка дворов или её отсутств...
7,6,30,6_дверь_двери_парадной_дверей,"[дверь, двери, парадной, дверей, доводчик, руч...",[какими должны быть входные двери подъездов в ...
8,7,29,7_собак_собака_собаки_собаку,"[собак, собака, собаки, собаку, это, площадке,...","[уважаемые соседи, жильцы арсенальной,3 срочна..."
9,8,29,8_ночи_спать_время_слышно,"[ночи, спать, время, слышно, утра, совесть, сп...","[дорогие соседи, помогите найти нарушителя ноч..."


In [27]:
topic_model_jkh_posts.get_representative_docs()

{-1: ['дорогие друзья, жители дома! эта группа создана в связи с тем, что любые альтернативные мнения удаляются с официальной страницы нашего тсж. к сожалению, наш дом находится в запущенном состоянии. по сравнению тем, что предоставил застройщик, ничего не изменилось!!! нет клумб, нет нормальных доводчиков на дверях парадных и т.п. деятельность тсж свелась к посредничеству по передаче наших денег поставляющим услуги организациям. то есть, они просто берут часть наших денег на свои личные нужды, и ничего не делают для благоустройства нашего дома. предлагаем обсудить перспективные направления развития и сохранения нашего дома!',
  'уважаемые жильцы лично я предлагаю 12 октября всем собраться на спортивной площадке в 14 ч и обсудить все происходящие, посмотрим сколько нас , составим коллективное письмо , текст тут набросаем, отнесём в правление дома с требованием собрать внеочередное собрание жильцов дома вместе с тсж и заслушать их отчёт 1 по ремонту дома, где был и зачем когда в холлах

In [28]:
topic_model_jkh_posts.save("jkh_groups_data/jkh_bertopic_posts2.pickle")

In [28]:
jkh_posts_df['topic'] = topics
jkh_posts_df.to_csv('jkh_groups_data/posts_with_topic.csv', index=None)

In [29]:
with open('jkh_groups_data/project-7-at-2023-05-01-12-54-a4f45cc1.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

temp = {k: [x[k] for x in data] for k in data[0].keys()}
jkh_prob_df = pd.DataFrame(temp)
jkh_prob_df['manual_label'] = [int(x.startswith('Есть')) for x in jkh_prob_df['sentiment']]
jkh_posts_df = jkh_posts_df.merge(jkh_prob_df[['post_id', 'owner_id', 'manual_label']], on=['post_id', 'owner_id'], how='left')
jkh_posts_df

,date,post_id,owner_id,text,clean_text,similar,neg_sent,has_phone,from_big_group,label,id,lat,lon,topic,manual_label
0,2018-11-23 00:00:00,33,-116263595,Добрый день. Напротив второй парадной у нас оч...,добрый день. напротив второй парадной у нас оч...,1,0,False,False,1,-116263595,59.938955,30.315644,2,0.0
1,2017-04-15 00:00:00,2269,-26534225,Середина апреля 2017 года наступила ...... Где...,середина апреля 2017 года наступила ...... где...,0,1,False,False,1,-26534225,60.012125,30.333951,-1,0.0
2,2017-05-17 00:00:00,2300,-26534225,2.Миф о невозможности заменить трубы - эта зим...,2.миф о невозможности заменить трубы - эта зим...,0,1,False,False,1,-26534225,60.012125,30.333951,0,0.0
3,2017-05-20 00:00:00,2307,-26534225,3. Результаты суда. На самом деле суд указал н...,3. результаты суда. на самом деле суд указал н...,0,1,False,False,1,-26534225,60.012125,30.333951,1,0.0
4,2017-05-23 00:00:00,2309,-26534225,4. Открытая информация и обсуждение всех! вопр...,4. открытая информация и обсуждение всех! вопр...,0,1,False,False,1,-26534225,60.012125,30.333951,4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,2021-09-22 20:18:28,208,-70134067,сумасшедший на уровне 9-10-11этажа 3й - 2й пар...,сумасшедший на уровне 9-10-11этажа 3й - 2й пар...,0,1,False,False,1,-70134067,59.937540,30.350687,5,NaN
1679,2020-12-10 09:51:26,129,-64635732,Уважаемые автовладельцы! \nДавайте не будем св...,уважаемые автовладельцы! давайте не будем свин...,0,1,False,False,1,-64635732,59.938955,30.315644,-1,NaN
1680,2021-04-12 12:41:40,173,-8210550,Открытие детского сада на 220 мест на ул. Черк...,открытие детского сада на 220 мест на ул. черк...,0,1,False,False,1,-8210550,60.030721,30.426199,-1,NaN
1681,2009-03-23 08:05:20,3,-8210550,"Да уж.....\nБеседа остановилась на том, что ad...","да уж..... беседа остановилась на том, что adm...",0,1,False,False,1,-8210550,60.030721,30.426199,-1,NaN


In [197]:
jkh_posts_df.to_csv('jkh_groups_data/jkh_posts2_final.csv', sep='|', index=None)

In [33]:
jkh_posts_df = pd.read_csv('jkh_groups_data/jkh_posts2_final.csv', sep='|')

In [ ]:
topic_model_jkh_comments = BERTopic.load("jkh_groups_data/jkh_bertopic_posts2.pickle")

# SemConvTree

Использую SemConvTree из неопубликованной на текущий момент статьи SemConvTree

In [35]:
jkh_docs = jkh_posts_df.clean_text.tolist()
jkh_embeddings = sentence_model.encode(jkh_docs, show_progress_bar=False)

topics, probs = topic_model_jkh_posts.transform(jkh_docs, jkh_embeddings)

In [36]:
probs_ = probs / np.linalg.norm(probs, ord=1, axis=-1)[..., np.newaxis]
probs_.sum()

1683.0

In [37]:
is_labelled = jkh_posts_df['manual_label'].notna().values
labels = jkh_posts_df[is_labelled]['manual_label'].values

In [72]:
# поиск с помощью GridSearch
# оцениваю насколько хорошо модель SemConvTree выделяет сообщения с проблемой лифта,
# то есть SemConvTree присваивает некоторый вес каждому сообщению, зачем этот вес по
# по порогу k переводится в 1 или 0, после чего вычисляется метрика F1.

TOPICS_TO_SEARCH = np.array([5])

df = jkh_posts_df
docs = jkh_docs

best_ver = None

results_dir = 'results_jkh'
os.makedirs(results_dir, exist_ok=True)

params_grid = {
    'n_range': [5, 10, 25, 50, 75, 100],
    'alpha': [0.1, 0.2, 0.5, 0.7, 1.0],
    'beta_topic': [1.0, 2.0, 3.0, 5.0, 10.0],
    'tpr_mode': ['year_month', 'year_3month'], #, 'busday'],
    'k': [1.5, 3.0, 5.0, 7.0]
#     'active_zone_threshold': [2.0, 5.0, 10.0],
#     'post_weight_threshold': [1.1, 1.5, 2.0],
}

def one_run(df, docs, probs, n_range, alpha, beta_topic, tpr_mode, path, k=5.0):
    geogrid = Geogrid(
        df['lat'].min(),
        df['lat'].max(),
        df['lon'].min(),
        df['lon'].max(),
        n_range=n_range,
    )

    tpr = TimePeriodRange(mode=tpr_mode)
    sct = SemConvTree(alpha, beta)
    
    sem_agg_topics = sct.transform(
        df[['lat', 'lon', 'date']].values,
        probs_, geogrid, tpr,
        return_active_zone_posts=False
    )
    
    all_posts_weights = sct.weighted_topics
    
    max_weight_posts_idxs = np.argpartition(all_posts_weights, -10)[-10:]
    min_weight_posts_idxs = np.argpartition(all_posts_weights, 10)[:10]
    
    norm_sem_agg_topics = sct.get_norm_sem_agg_topics()
    sem_agg_topics = sct.get_sem_agg_topics()
    lat_idxs, lon_idxs, time_idxs = sct.get_posts_idxs()
    
    active_zones_idxs = np.argsort(sem_agg_topics, axis=None)
    active_zones_idxs = active_zones_idxs[-50:]
    threshold = np.sort(sem_agg_topics.flat[active_zones_idxs])[-10]
    active_zones_idxs = active_zones_idxs[sem_agg_topics.flat[active_zones_idxs] > threshold]
    
    os.makedirs(path, exist_ok=True)
    with open(f'{path}/config.json', 'w', encoding='utf-8') as f:
        json.dump(params, f)
        
    with open(f'{path}/max_weights_posts.txt', 'w', encoding='utf-8') as f:
        f.write('\n'.join(map(lambda x: f'{x} | {np.round(all_posts_weights[x], 3)} | {docs[x]}', max_weight_posts_idxs)))
        
    with open(f'{path}/min_weights_posts.txt', 'w', encoding='utf-8') as f:
        f.write('\n'.join(map(lambda x: f'{x} | {np.round(all_posts_weights[x], 3)} | {docs[x]}', min_weight_posts_idxs)))
    
    
    with open(f'{path}/active_zones.txt', 'w', encoding='utf-8') as f:
        f.write('\n'.join(map(lambda x: f'{x} {sem_agg_topics.flat[x]}', active_zones_idxs)))
        
    os.makedirs(f'{path}/active_zone_posts', exist_ok=True)
    for idx in active_zones_idxs:
        in_active_zone_fn = lambda x: x[1][0]*tpr.n_range*n_range+x[1][1]*tpr.n_range+x[1][2] == idx
        posts_idxs_in_zone = filter(in_active_zone_fn, enumerate(zip(lat_idxs, lon_idxs, time_idxs)))
        with open(f'{path}/active_zone_posts/{idx}', 'w', encoding='utf-8') as f:
            f.write('\n'.join(map(lambda x: f'{x[0]} | {np.round(all_posts_weights[x[0]], 3)} | {docs[x[0]]}', posts_idxs_in_zone)))
    
    algo_res_for_metrics = (all_posts_weights[is_labelled] > k).astype(int)
    
    metrics = {}
    metrics['avg_posts_count_in_cell'] = len(df) / len(set(zip(lat_idxs, lon_idxs, time_idxs)))
    metrics['recall'] = recall_score(labels, algo_res_for_metrics)
    metrics['precision'] = precision_score(labels, algo_res_for_metrics)
    metrics['f1'] = f1_score(labels, algo_res_for_metrics)
    
    with open(f'{path}/metrics.json', 'w') as f:
        json.dump(metrics, f)

for ver, params in enumerate(ParameterGrid(params_grid)):
    # GeoGrid params
    n_range = params['n_range']
    # SemConvTree params
    alpha = params['alpha']
    beta = np.zeros(probs_.shape[1])
    for topic in TOPICS_TO_SEARCH:
        beta[topic] = params['beta_topic']
    tpr_mode = params['tpr_mode']
    k = params['k']
#     active_zone_threshold = params['active_zone_threshold']
#     post_weight_threshold = params['post_weight_threshold']
    
    path = f'{results_dir}/v{ver:03d}'
    one_run(df, docs, probs_, n_range, alpha, beta, tpr_mode, path, k=k)

In [73]:
metrics_data = defaultdict(list)

for ver in os.listdir('results_jkh'):
    metrics_data['ver'].append(ver)
    with open(f'results_jkh/{ver}/metrics.json', 'r') as f:
        d = json.load(f)
    for k in d:
        metrics_data[k].append(d[k])
    with open(f'results_jkh/{ver}/config.json', 'r') as f:
        d = json.load(f)
    for k in d:
        metrics_data[k].append(d[k])

metrics_df = pd.DataFrame(metrics_data)
metrics_df = metrics_df.sort_values(['f1'], ascending=False)

In [74]:
metrics_df

,ver,avg_posts_count_in_cell,recall,precision,f1,alpha,beta_topic,k,n_range,tpr_mode
1155,v112,2.047445,0.2,0.269231,0.229508,0.1,3.0,3.0,25,year_month
642,v064,2.047445,0.2,0.269231,0.229508,0.1,2.0,3.0,25,year_month
536,v435,8.904762,0.4,0.159091,0.227642,0.2,10.0,1.5,10,year_3month
759,v387,8.904762,0.4,0.159091,0.227642,0.2,5.0,1.5,10,year_3month
693,v291,8.904762,0.4,0.157303,0.225806,0.2,2.0,1.5,10,year_3month
...,...,...,...,...,...,...,...,...,...,...
457,v613,13.795082,0.0,0.000000,0.000000,0.5,3.0,7.0,5,year_3month
458,v1096,2.047445,0.0,0.000000,0.000000,1.0,3.0,7.0,25,year_month
461,v795,8.904762,0.0,0.000000,0.000000,0.7,2.0,5.0,10,year_3month
462,v660,4.822350,0.0,0.000000,0.000000,0.5,5.0,7.0,5,year_month


In [81]:
k = 7.0
metrics_df[metrics_df['k'] == k]

,ver,avg_posts_count_in_cell,recall,precision,f1,alpha,beta_topic,k,n_range,tpr_mode
786,v088,2.047445,0.085714,1.00,0.157895,0.1,2.0,7.0,25,year_month
616,v184,2.047445,0.085714,1.00,0.157895,0.1,5.0,7.0,25,year_month
388,v232,2.047445,0.085714,1.00,0.157895,0.1,10.0,7.0,25,year_month
559,v136,2.047445,0.085714,1.00,0.157895,0.1,3.0,7.0,25,year_month
495,v040,2.047445,0.085714,0.75,0.153846,0.1,1.0,7.0,25,year_month
...,...,...,...,...,...,...,...,...,...,...
453,v189,3.105166,0.000000,0.00,0.000000,0.1,5.0,7.0,75,year_3month
457,v613,13.795082,0.000000,0.00,0.000000,0.5,3.0,7.0,5,year_3month
458,v1096,2.047445,0.000000,0.00,0.000000,1.0,3.0,7.0,25,year_month
462,v660,4.822350,0.000000,0.00,0.000000,0.5,5.0,7.0,5,year_month


## Веса сообщений для каждого топика

то есть выставляется внимание каждый раз только на один топик

In [91]:
def one_run_2(df, probs, n_range, alpha, beta, tpr_mode):
    geogrid = Geogrid(
        df['lat'].min(),
        df['lat'].max(),
        df['lon'].min(),
        df['lon'].max(),
        n_range=n_range,
    )

    tpr = TimePeriodRange(mode=tpr_mode)
    sct = SemConvTree(alpha, beta)
    
    _ = sct.transform(
        df[['lat', 'lon', 'date']].values,
        probs, geogrid, tpr,
        return_active_zone_posts=False
    )
    
    all_posts_weights = sct.weighted_topics
    
    return all_posts_weights

In [49]:
best_ver = metrics_df.head(1)['ver'].values[0]
with open(f'results_jkh/{best_ver}/config.json', 'r') as f:
    best_params = json.load(f)

for topic in range(1, probs_.shape[1]):
    n_range = best_params['n_range']
    # SemConvTree params
    alpha = best_params['alpha']
    beta = np.zeros(probs_.shape[1])
    beta[topic] = best_params['beta_topic']
    tpr_mode = best_params['tpr_mode']
    
    posts_weights = one_run_2(df, probs_, n_range, alpha, beta, tpr_mode)
    np.save(f'jkh_groups_data/specific_topic/{topic}.npy', posts_weights)

## Вычисляю веса уже для сообщений до фильтрации с помощью GPT

In [50]:
posts2_df = pd.read_csv('jkh_groups_data/candidates_posts_labelled.csv', sep='|')
posts2_df

,date,post_id,owner_id,text,clean_text,similar,neg_sent,has_phone,from_big_group,label
0,2018-11-23 00:00:00,33,-116263595,Добрый день. Напротив второй парадной у нас оч...,добрый день. напротив второй парадной у нас оч...,1,0,False,False,1
1,2017-04-15 00:00:00,2269,-26534225,Середина апреля 2017 года наступила ...... Где...,середина апреля 2017 года наступила ...... где...,0,1,False,False,1
2,2017-05-15 00:00:00,2296,-26534225,В преддверии годового собрания ТСЖ хотелось бы...,в преддверии годового собрания тсж хотелось бы...,0,1,False,False,0
3,2017-05-17 00:00:00,2300,-26534225,2.Миф о невозможности заменить трубы - эта зим...,2.миф о невозможности заменить трубы - эта зим...,0,1,False,False,1
4,2017-05-20 00:00:00,2307,-26534225,3. Результаты суда. На самом деле суд указал н...,3. результаты суда. на самом деле суд указал н...,0,1,False,False,1
...,...,...,...,...,...,...,...,...,...,...
2830,2009-03-22 12:14:52,2,-8210550,Сайт УЧИТЕЛЬ закрыт для неугодных...) Скопир...,сайт учитель закрыт для неугодных... скопирова...,0,1,False,False,0
2831,2020-03-18 07:03:27,623,-13674650,Максимальный репост. Президент группы компаний...,максимальный репост. президент группы компаний...,0,1,False,False,0
2832,2015-05-08 22:31:23,456,-13674650,Дорогие ветераны Великой Отечественной войны и...,дорогие ветераны великой отечественной войны и...,0,1,False,False,0
2833,2015-04-15 12:53:06,449,-13674650,Информация для жителей Невского района!\nБудьт...,информация для жителей невского района! будьте...,0,1,False,False,1


In [16]:
topic_model_jkh_posts = BERTopic.load("jkh_groups_data/jkh_bertopic_posts")

In [88]:
posts_df = pd.read_csv('jkh_groups_data/candidates_posts3.csv', sep='|')
posts2_df = pd.read_csv('jkh_groups_data/candidates_posts_labelled.csv', sep='|')
posts2_df = posts2_df[posts2_df.columns[:-1]]

groups_temp_df = pd.read_csv('jkh_groups_data/vk_groups_addresses2.csv', usecols=['id', 'lat', 'lon']).drop_duplicates('id')
groups_temp_df['id'] = -groups_temp_df['id']

posts2_df = posts2_df.merge(groups_temp_df, left_on='owner_id', right_on='id', how='left').drop(columns='id')

posts_df = pd.concat([posts2_df, posts_df])

posts_docs = posts_df.clean_text.tolist()
posts_embeddings = sentence_model.encode(posts_docs, show_progress_bar=False)

_, posts_probs = topic_model_jkh_posts.transform(posts_docs, posts_embeddings)

posts_probs_ = posts_probs / np.linalg.norm(posts_probs, ord=1, axis=-1)[..., np.newaxis]

In [245]:
posts_df.to_csv('jkh_groups_data/candidates_posts_with_3.csv', sep='|', index=None)

In [92]:
best_ver = metrics_df.head(1)['ver'].values[0]
with open(f'results_jkh/{best_ver}/config.json', 'r') as f:
    best_params = json.load(f)

for topic in range(probs_.shape[1]):
    n_range = best_params['n_range']
    # SemConvTree params
    alpha = best_params['alpha']
    beta = np.zeros(probs_.shape[1])
    beta[topic] = best_params['beta_topic']
    tpr_mode = best_params['tpr_mode']
    
    posts_weights = one_run_2(posts_df, posts_probs_, n_range, alpha, beta, tpr_mode)
    np.save(f'jkh_groups_data/specific_topic2/{topic}.npy', posts_weights)

In [ ]:
# здесь внимание выставляется сразу нескольким топикам, потенциально их надо вручную выбрать
# так, чтобы среди них были только топики с ЖКХ проблемами

TOPICS_TO_SEARCH = [0, 4, 6, 13, 12, 15, 16, 23, 24]

n_range = best_params['n_range']
# SemConvTree params
alpha = best_params['alpha']
beta = np.zeros(posts_probs_.shape[1])
for topic in TOPICS_TO_SEARCH:
    beta[topic] = best_params['beta_topic']
tpr_mode = best_params['tpr_mode']

posts_weights = one_run_2(posts_df, posts_probs_, n_range, alpha, beta, tpr_mode)
np.save(f'jkh_groups_data/specific_topic2/all_posts_with_3.npy', posts_weights)

In [223]:
vk_posts3_df = pd.read_csv('jkh/jkh_posts3_processed.csv')

not_empty = vk_posts3_df[vk_posts3_df['clean_text'].str.strip().str.len() > 0]
filtered = not_empty[((not_empty['similar'] == 1) | (not_empty['neg_sent'] == 1)) & (~not_empty['has_phone']) & (~not_empty['from_big_group'])]
print(len(filtered))

filtered.to_csv('jkh_groups_data/candidates_posts3.csv', sep='|', index=None)

2349


# Проверка

проверяю отдельные закупки на то, какие сообщения были до закупки и после

In [82]:
timestamps = ['2021-02-10', '2022-01-27', '2019-09-19', '2020-11-20', '2020-11-30', '2021-06-16', '2021-03-15', '2022-11-07', '2022-07-19']
lats = [59.943214, 59.943214, 59.926575, 59.940001, 59.92267, 60.006152, 60.009107, 59.833072, 59.918756]
lons = [30.479442, 30.479442, 30.377933, 30.469956, 30.448792, 30.252429, 30.24898, 30.312329, 30.469184]

timestamps = [datetime.datetime.strptime(ts,'%Y-%m-%d') for ts in timestamps]

In [85]:
posts_df = pd.read_csv('jkh_groups_data/candidates_posts_with_3.csv', sep='|')

In [86]:
best_ver = metrics_df.head(1)['ver'].values[0]
with open(f'results_jkh/{best_ver}/config.json', 'r') as f:
    best_params = json.load(f)

n_range = best_params['n_range']
tpr_mode = best_params['tpr_mode']

df = posts_df

geogrid = Geogrid(
    df['lat'].min(),
    df['lat'].max(),
    df['lon'].min(),
    df['lon'].max(),
    n_range=n_range,
)

tpr = TimePeriodRange(mode=tpr_mode)

test_geo_idxs = geogrid.get_grid_idxs(lats, lons)
test_tpr_idxs = tpr.get_range_idxs(timestamps)

posts_geo_idxs = geogrid.get_grid_idxs(df['lat'], df['lon'])
posts_tpr_idxs = tpr.get_range_idxs(df['date'])

In [101]:
before_offset = 1
after_offset = 1
min_change = 0.2

for cntr in range(len(timestamps)):
    geo_cond = (posts_geo_idxs[0] == test_geo_idxs[0][cntr]) & (posts_geo_idxs[1] == test_geo_idxs[1][cntr])
        
    before_cond = (test_tpr_idxs[cntr] - before_offset <= posts_tpr_idxs) & (test_tpr_idxs[cntr] >= posts_tpr_idxs)
    after_cond = (test_tpr_idxs[cntr] <= posts_tpr_idxs) & (test_tpr_idxs[cntr] + after_offset >= posts_tpr_idxs)

    print(f'{timestamps[cntr]=} | {lats[cntr]=} | {lons[cntr]=}')
    for topic in range(0, posts_probs_.shape[1]):
        posts_weights = np.load(f'jkh_groups_data/specific_topic2/{topic}.npy')
        
        before_mean = posts_weights[geo_cond & before_cond].mean()
        after_mean = posts_weights[geo_cond & after_cond].mean()
        if abs(before_mean - after_mean) > min_change:
            print(f"{topic=} | {(geo_cond & before_cond).sum()=} | {(geo_cond & after_cond).sum()=} | {before_mean=:0.3f} | {after_mean=:0.3f}")
    
    posts_weights = np.load(f'jkh_groups_data/specific_topic2/all_posts_with_3.npy')
        
    before_mean = posts_weights[geo_cond & before_cond].mean()
    after_mean = posts_weights[geo_cond & after_cond].mean()
    if before_mean > after_mean + 0.2:
        print(f"all_posts_with_3 | {(geo_cond & before_cond).sum()=} | {(geo_cond & after_cond).sum()=} | {before_mean=:0.3f} | {after_mean=:0.3f}")
    
    print('-'*100)

timestamps[cntr]=datetime.datetime(2021, 2, 10, 0, 0) | lats[cntr]=59.943214 | lons[cntr]=30.479442
topic=36 | (geo_cond & before_cond).sum()=22 | (geo_cond & after_cond).sum()=25 | before_mean=1.125 | after_mean=0.822
----------------------------------------------------------------------------------------------------
timestamps[cntr]=datetime.datetime(2022, 1, 27, 0, 0) | lats[cntr]=59.943214 | lons[cntr]=30.479442
topic=4 | (geo_cond & before_cond).sum()=24 | (geo_cond & after_cond).sum()=28 | before_mean=0.870 | after_mean=0.645
topic=6 | (geo_cond & before_cond).sum()=24 | (geo_cond & after_cond).sum()=28 | before_mean=0.874 | after_mean=0.640
topic=10 | (geo_cond & before_cond).sum()=24 | (geo_cond & after_cond).sum()=28 | before_mean=1.149 | after_mean=0.920
topic=12 | (geo_cond & before_cond).sum()=24 | (geo_cond & after_cond).sum()=28 | before_mean=0.941 | after_mean=0.706
topic=14 | (geo_cond & before_cond).sum()=24 | (geo_cond & after_cond).sum()=28 | before_mean=0.939 | afte